Replicate [Dynamic Return Dependencies Across Industries: A Machine Learning Approach](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3120110&download=yes) by David Rapach, Jack Strauss, Jun Tu and Guofu Zhou.

1) Use industry returns from [Ken French](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html)

2) Forecast (for example) this month's Chemical industry return using last month's returns from all 30 industries 

3) Use LASSO for predictor subset selection over the entire 1960-2016 period to determine that e.g. Beer is predicted by Food, Clothing, Coal

4) Use those predictors and simple linear regression to predict returns

5) Generate portfolios and run backtests.

- Predictor selection - finds same predictors except 2 industries. Possibly use of AICc instead of AIC (don't see an sklearn implementation that uses AICc)

- Prediction by industry - R-squareds line up pretty closely

- Portfolio performance, similar ballpark results. Since prediction is similar but return profile is different, must be some difference in portfolio construction. (am taking equal weight top 6 predicted as long and bottom 6 as short, every month)

- For some reason their mean returns don't line up to geometric mean annualized, they seem to be calculating something different.

- But it does replicate closely and perform pretty well

In [82]:
import os
import sys
import warnings
import numpy as np
import pandas as pd
import pandas_datareader.data as datareader
import time 
import datetime
import copy
import random
from itertools import product

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") #Hide messy numpy warnings

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, explained_variance_score, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler


import tensorflow as tf
tf.set_random_seed(1764)

import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Input
from keras.models import Model

from keras.layers.recurrent import LSTM, GRU
from keras.regularizers import l1
from keras.models import Sequential
from keras.models import load_model

import ffn
%matplotlib inline

import plotly as py
# print (py.__version__) # requires version >= 1.9.0
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.figure_factory as ff

init_notebook_mode(connected=True)

random.seed(1764)
np.random.seed(1764)


In [83]:
print("Loading data...")
data = pd.read_csv("30_Industry_Portfolios.csv")
data = data.set_index('yyyymm')
industries = list(data.columns)
# map industry names to col nums
ind_reverse_dict = dict([(industries[i], i) for i in range(len(industries))])

rfdata = pd.read_csv("F-F_Research_Data_Factors.csv")
rfdata = rfdata.set_index('yyyymm')
data['rf'] = rfdata['RF']

# subtract risk-free rate
# create a response variable led by 1 period to predict
for ind in industries:
    data[ind] = data[ind] - data['rf']

    
# add rates data from FRED
start_date = datetime.datetime(1926, 9, 1)
end_date = datetime.datetime(2017, 12, 1)
TB3MS = datareader.DataReader("TB3MS", "fred", start_date, end_date)
TB3MS['yyyymm'] = TB3MS.index.strftime('%Y%m')
TB3MS['yyyymm'] = [int(datestr) for datestr in TB3MS['yyyymm']]
TB3MS=TB3MS.set_index(['yyyymm'])
data['3month']=TB3MS['TB3MS']

GS10 =  datareader.DataReader("GS10", "fred", start_date, end_date)
GS10['yyyymm'] = GS10.index.strftime('%Y%m')
GS10['yyyymm'] = [int(datestr) for datestr in GS10['yyyymm']]
GS10=GS10.set_index(['yyyymm'])
data['10year']=GS10['GS10']

data['curve'] = data['10year'] - data['3month']
data['10year'] = data['10year'].diff() # first difference 10-year yield
data['3month'] = data['3month'].diff() # first difference 3-month
data['month'] = (data.index  % 100)/12.0 # for possible seasonality
data['Mkt-RF'] = rfdata['Mkt-RF']

for ind in industries + ['3month', '10year', 'curve', 'Mkt-RF',]:
    data[ind+".3m"] = pd.rolling_mean(data[ind],3)
    
#for ind in industries + ['3month', '10year', 'curve', 'Mkt-RF',]:
#    data[ind+".6m"] = pd.rolling_mean(data[ind],6)

for ind in industries + ['3month', '10year', 'curve', 'Mkt-RF',]:
    data[ind+".12m"] = pd.rolling_mean(data[ind],12)

for ind in industries:
    data[ind+".lead"] = data[ind].shift(-1)

data = data.loc[data.index[data.index > 195911]]
data = data.drop(columns=['rf'])    
data = data.dropna(axis=0, how='any')

nresponses = len(industries)
npredictors = data.shape[1]-nresponses

predictors = list(data.columns[:npredictors])
predictor_reverse_dict = dict([(predictors[i], i) for i in range(len(predictors))])

responses = list(data.columns[-nresponses:])
response_reverse_dict = dict([(responses[i], i) for i in range(len(responses))])

print(data.shape)
print(list(data.columns))
data[['3month', '10year', 'curve', 'month', 'Mkt-RF',]]

Loading data...
(697, 133)
['Food', 'Beer', 'Smoke', 'Games', 'Books', 'Hshld', 'Clths', 'Hlth', 'Chems', 'Txtls', 'Cnstr', 'Steel', 'FabPr', 'ElcEq', 'Autos', 'Carry', 'Mines', 'Coal', 'Oil', 'Util', 'Telcm', 'Servs', 'BusEq', 'Paper', 'Trans', 'Whlsl', 'Rtail', 'Meals', 'Fin', 'Other', '3month', '10year', 'curve', 'month', 'Mkt-RF', 'Food.3m', 'Beer.3m', 'Smoke.3m', 'Games.3m', 'Books.3m', 'Hshld.3m', 'Clths.3m', 'Hlth.3m', 'Chems.3m', 'Txtls.3m', 'Cnstr.3m', 'Steel.3m', 'FabPr.3m', 'ElcEq.3m', 'Autos.3m', 'Carry.3m', 'Mines.3m', 'Coal.3m', 'Oil.3m', 'Util.3m', 'Telcm.3m', 'Servs.3m', 'BusEq.3m', 'Paper.3m', 'Trans.3m', 'Whlsl.3m', 'Rtail.3m', 'Meals.3m', 'Fin.3m', 'Other.3m', '3month.3m', '10year.3m', 'curve.3m', 'Mkt-RF.3m', 'Food.12m', 'Beer.12m', 'Smoke.12m', 'Games.12m', 'Books.12m', 'Hshld.12m', 'Clths.12m', 'Hlth.12m', 'Chems.12m', 'Txtls.12m', 'Cnstr.12m', 'Steel.12m', 'FabPr.12m', 'ElcEq.12m', 'Autos.12m', 'Carry.12m', 'Mines.12m', 'Coal.12m', 'Oil.12m', 'Util.12m', 'Telcm.1

,3month,10year,curve,month,Mkt-RF
yyyymm,,,,,
195912,0.34,0.16,0.20,1.000000,2.45
196001,-0.14,0.03,0.37,0.083333,-6.98
196002,-0.39,-0.23,0.53,0.166667,1.17
196003,-0.65,-0.24,0.94,0.250000,-1.63
196004,-0.08,0.03,1.05,0.333333,-1.71
196005,0.06,0.07,1.06,0.416667,3.12
196006,-0.83,-0.20,1.69,0.500000,2.08
196007,-0.16,-0.25,1.60,0.583333,-2.37
196008,0.00,-0.10,1.50,0.666667,3.01


In [84]:
#data = data.loc[data.index[data.index < 201701]]
data = data.loc[data.index[data.index > 195911]]
data


,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
yyyymm,,,,,,,,,,,,,,,,,,,,,
195912,2.01,0.35,-3.02,1.64,7.29,0.67,1.87,-1.97,3.08,0.74,...,0.62,-6.18,-7.93,-9.41,-4.31,-5.33,-6.09,-10.08,-4.68,-3.98
196001,-4.49,-5.71,-2.05,1.21,-5.47,-7.84,-8.53,-6.68,-10.03,-4.77,...,8.07,9.13,5.09,3.00,-0.94,1.42,4.00,1.81,-0.98,6.32
196002,3.35,-2.14,2.27,4.23,2.39,9.31,1.44,-0.02,-0.74,0.32,...,-0.21,-0.31,3.34,-2.43,-4.99,-1.37,-0.13,-3.88,0.05,-2.43
196003,-1.67,-2.94,-0.18,-0.65,2.18,-0.56,-2.59,1.26,-2.75,-6.79,...,-1.24,7.14,1.77,0.41,-2.13,0.45,-0.53,8.86,-0.64,0.55
196004,1.17,-2.16,1.35,6.46,-1.17,-1.27,0.21,1.49,-5.53,-1.10,...,3.05,-1.75,11.90,2.85,0.90,1.65,3.11,0.80,-0.45,1.02
196005,8.20,-0.52,2.44,7.28,11.67,7.74,1.74,13.50,3.40,2.10,...,-0.58,-8.07,2.39,3.50,2.17,5.96,3.41,1.03,3.72,6.41
196006,5.39,0.47,4.73,2.24,0.02,6.38,-1.59,-0.40,0.45,4.04,...,-0.03,2.84,-2.02,-4.10,-3.11,-6.16,-2.99,-1.25,0.09,-5.95
196007,-2.11,-0.79,4.60,-4.72,0.23,-0.60,-1.10,-3.99,-6.80,-3.14,...,6.94,5.69,2.71,1.18,1.98,4.51,2.85,2.05,3.47,3.48
196008,4.57,3.24,5.20,7.16,3.63,5.09,3.34,2.29,1.17,-0.84,...,-6.07,-3.53,-7.61,-7.37,-7.07,-8.44,-8.57,-1.90,-5.78,-4.21


In [85]:
desc = data.describe()
desc
# min, max line up with Table 1

,Food,Beer,Smoke,Games,Books,Hshld,Clths,Hlth,Chems,Txtls,...,Telcm.lead,Servs.lead,BusEq.lead,Paper.lead,Trans.lead,Whlsl.lead,Rtail.lead,Meals.lead,Fin.lead,Other.lead
count,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,...,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000,697.000000
mean,0.688666,0.727030,0.985079,0.732095,0.532253,0.564333,0.690387,0.665825,0.552367,0.687145,...,0.515968,0.729928,0.622970,0.534806,0.601090,0.631076,0.698235,0.728766,0.637547,0.396628
std,4.308660,5.058992,6.032324,7.128170,5.780362,4.728000,6.355251,4.897557,5.482363,6.970961,...,4.607931,6.486956,6.698787,5.021876,5.707154,5.571040,5.334178,6.065564,5.381389,5.771655
min,-18.150000,-20.190000,-25.320000,-33.400000,-26.560000,-22.240000,-31.500000,-21.060000,-28.600000,-33.110000,...,-16.440000,-28.670000,-32.070000,-27.740000,-28.500000,-29.250000,-29.740000,-31.890000,-22.530000,-28.090000
25%,-1.630000,-2.080000,-2.740000,-3.390000,-2.600000,-2.030000,-2.800000,-2.230000,-2.750000,-3.170000,...,-2.110000,-3.050000,-3.220000,-2.400000,-2.780000,-2.560000,-2.380000,-2.840000,-2.400000,-2.930000
50%,0.740000,0.750000,1.270000,0.940000,0.510000,0.750000,0.700000,0.760000,0.720000,0.640000,...,0.590000,1.010000,0.670000,0.710000,0.900000,0.940000,0.540000,1.080000,0.870000,0.540000
75%,3.070000,3.690000,4.660000,5.260000,3.640000,3.540000,4.310000,3.550000,3.760000,4.480000,...,3.360000,4.260000,4.630000,3.460000,4.040000,3.880000,3.980000,4.300000,4.000000,4.200000
max,19.890000,25.510000,32.380000,34.520000,33.130000,18.220000,31.790000,29.010000,21.680000,59.030000,...,21.220000,23.380000,24.660000,21.000000,18.500000,17.530000,26.490000,27.380000,20.590000,19.960000


In [86]:
# Run LASSO, then OLS on selected variables

# skip last row to better match published r-squared
# looks like they forecast actuals 1960-2016 using 1959m12 to 2016m11
# not exact matches to Table 2 R-squared but almost within rounding error 
X = data.values[:-1,:npredictors]
Y = data.values[:-1,-nresponses:]
nrows = X.shape[0]
X.shape

(696, 103)

In [87]:
# convert Ys to 3 classes
# long = 1
# short = 2
# neither = 0
ISLONG=1
ISSHORT=2
ISFLAT=0

Y_sortindex = np.argsort(Y)
print(Y[0])
# sorted position
print(Y_sortindex[0]) 
# sorted array
print(Y[0,Y_sortindex[0]])
# initialize class to 0
Y_class=np.zeros_like(Y)
for row in range(Y_class.shape[0]):
    # if index in last 6, long
    longlist = Y_sortindex[row,-6:]
    Y_class[row, longlist]=ISLONG
    # if index is in first 6, short
    shortlist = Y_sortindex[row,:6]
    Y_class[row, shortlist]=ISSHORT
    
print(Y_class[0])
print([Y[0,i] for i in range(30) if Y_class[0,i]==1])
print([Y[0,i] for i in range(30) if Y_class[0,i]==-1])
print(Y_class.shape)
print(X.shape)


[ -4.49  -5.71  -2.05   1.21  -5.47  -7.84  -8.53  -6.68 -10.03  -4.77
  -6.67  -9.38  -4.42 -12.3  -11.71  -5.03  -3.81  -7.91  -7.82  -2.4
   0.62  -6.18  -7.93  -9.41  -4.31  -5.33  -6.09 -10.08  -4.68  -3.98]
[13 14 27  8 23 11  6 22 17  5 18  7 10 21 26  1  4 25 15  9 28  0 12 24
 29 16 19  2 20  3]
[-12.3  -11.71 -10.08 -10.03  -9.41  -9.38  -8.53  -7.93  -7.91  -7.84
  -7.82  -6.68  -6.67  -6.18  -6.09  -5.71  -5.47  -5.33  -5.03  -4.77
  -4.68  -4.49  -4.42  -4.31  -3.98  -3.81  -2.4   -2.05   0.62   1.21]
[0. 0. 1. 1. 0. 0. 0. 0. 2. 0. 0. 2. 0. 2. 2. 0. 1. 0. 0. 1. 1. 0. 0. 2.
 0. 0. 0. 2. 0. 1.]
[-2.05, 1.21, -3.81, -2.4, 0.6199999999999999, -3.98]
[]
(696, 30)
(696, 103)


In [88]:
INPUT_DIM = X.shape[1]
print(INPUT_DIM)
OUTPUT_DIM = len(responses) # 30
NCLASSES=3
EPOCHS=100
BATCH_SIZE=32

def build_model(n_hidden_layers = 2,
                hidden_layer_size = 32,
                reg_penalty = 0.0001,
                dropout = 0.25,
                verbose=True):

    main_input = Input(shape=(INPUT_DIM,), 
                       dtype='float32', 
                       name='main_input')
    lastlayer=main_input

    for i in range(n_hidden_layers):
        if verbose:
            print("layer %d size %d, reg_penalty %.8f, dropout %.3f" % (i+1, hidden_layer_size, reg_penalty, dropout))
        lastlayer = Dense(units = hidden_layer_size, 
                          activation = 'relu',
                          kernel_initializer = keras.initializers.glorot_uniform(),
                          kernel_regularizer=keras.regularizers.l1(reg_penalty),
                          name = "Dense%02d" % (i+1))(lastlayer)

        if dropout:
            lastlayer = Dropout(dropout, name = "Dropout%02d" % i)(lastlayer)

    outputs=[]
    for i in range(OUTPUT_DIM):
        outputs.append (Dense(NCLASSES, 
                              activation='softmax',
                              name = "Output%02d" % (i+1))(lastlayer))
    
    model = Model(inputs=[main_input], outputs=outputs)
    if verbose:
        print(model.summary())
    model.compile(loss="categorical_crossentropy", 
                  optimizer="rmsprop", 
                  metrics=['accuracy'])
    return model

# Convert labels to categorical one-hot encoding
responselist=[]
for i in range(OUTPUT_DIM):
    one_hot_labels = keras.utils.to_categorical(Y_class[:,i], num_classes=NCLASSES)
    responselist.append(one_hot_labels)
    
# Train the model
model = build_model()
model.fit(X, responselist, epochs=EPOCHS, batch_size=BATCH_SIZE)


103
layer 1 size 32, reg_penalty 0.00010000, dropout 0.333
layer 2 size 32, reg_penalty 0.00010000, dropout 0.333
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 103)          0                                            
__________________________________________________________________________________________________
Dense01 (Dense)                 (None, 32)           3328        main_input[0][0]                 
__________________________________________________________________________________________________
Dropout00 (Dropout)             (None, 32)           0           Dense01[0][0]                    
__________________________________________________________________________________________________
Dense02 (Dense)                 (None, 32)           1056        Dropout00[0][0]              

696/696 [==============================] - 5s 7ms/step - loss: 72.5101 - Output01_loss: 2.4124 - Output02_loss: 2.1713 - Output03_loss: 2.1698 - Output04_loss: 2.4353 - Output05_loss: 2.1508 - Output06_loss: 2.8134 - Output07_loss: 2.3008 - Output08_loss: 2.7573 - Output09_loss: 2.8213 - Output10_loss: 2.3184 - Output11_loss: 1.9868 - Output12_loss: 2.1085 - Output13_loss: 2.1920 - Output14_loss: 1.7980 - Output15_loss: 3.0812 - Output16_loss: 2.9140 - Output17_loss: 2.2914 - Output18_loss: 2.8460 - Output19_loss: 3.1521 - Output20_loss: 2.3393 - Output21_loss: 3.3997 - Output22_loss: 2.1212 - Output23_loss: 2.9919 - Output24_loss: 1.7213 - Output25_loss: 1.8332 - Output26_loss: 1.9838 - Output27_loss: 2.1681 - Output28_loss: 2.5474 - Output29_loss: 2.0453 - Output30_loss: 2.5887 - Output01_acc: 0.3118 - Output02_acc: 0.3678 - Output03_acc: 0.3736 - Output04_acc: 0.3132 - Output05_acc: 0.3951 - Output06_acc: 0.2730 - Output07_acc: 0.4167 - Output08_acc: 0.3348 - Output09_acc: 0.2802 - 

696/696 [==============================] - 2s 3ms/step - loss: 33.3468 - Output01_loss: 1.0784 - Output02_loss: 1.1506 - Output03_loss: 1.2420 - Output04_loss: 1.2229 - Output05_loss: 1.0873 - Output06_loss: 1.0671 - Output07_loss: 1.0958 - Output08_loss: 1.1648 - Output09_loss: 1.0512 - Output10_loss: 1.1724 - Output11_loss: 0.9743 - Output12_loss: 1.1437 - Output13_loss: 0.9849 - Output14_loss: 1.0587 - Output15_loss: 1.2056 - Output16_loss: 1.1096 - Output17_loss: 1.2633 - Output18_loss: 1.2486 - Output19_loss: 1.1956 - Output20_loss: 1.1937 - Output21_loss: 1.1497 - Output22_loss: 1.1324 - Output23_loss: 1.1835 - Output24_loss: 0.8789 - Output25_loss: 1.0481 - Output26_loss: 0.9542 - Output27_loss: 1.0433 - Output28_loss: 1.1477 - Output29_loss: 1.0179 - Output30_loss: 1.0331 - Output01_acc: 0.5575 - Output02_acc: 0.4555 - Output03_acc: 0.3707 - Output04_acc: 0.4224 - Output05_acc: 0.5647 - Output06_acc: 0.5402 - Output07_acc: 0.4986 - Output08_acc: 0.4655 - Output09_acc: 0.5503 - 

696/696 [==============================] - 2s 3ms/step - loss: 29.7827 - Output01_loss: 0.9248 - Output02_loss: 1.0548 - Output03_loss: 1.1099 - Output04_loss: 1.1013 - Output05_loss: 0.9423 - Output06_loss: 0.9588 - Output07_loss: 1.0162 - Output08_loss: 0.9895 - Output09_loss: 0.8921 - Output10_loss: 1.0671 - Output11_loss: 0.8381 - Output12_loss: 1.0599 - Output13_loss: 0.9337 - Output14_loss: 0.9661 - Output15_loss: 1.0656 - Output16_loss: 0.9783 - Output17_loss: 1.1420 - Output18_loss: 1.1452 - Output19_loss: 1.0795 - Output20_loss: 1.0769 - Output21_loss: 1.0173 - Output22_loss: 1.0199 - Output23_loss: 1.0309 - Output24_loss: 0.7797 - Output25_loss: 0.9545 - Output26_loss: 0.8246 - Output27_loss: 0.9414 - Output28_loss: 1.0059 - Output29_loss: 0.9119 - Output30_loss: 0.9081 - Output01_acc: 0.6437 - Output02_acc: 0.5244 - Output03_acc: 0.3851 - Output04_acc: 0.4454 - Output05_acc: 0.6365 - Output06_acc: 0.6523 - Output07_acc: 0.5718 - Output08_acc: 0.5747 - Output09_acc: 0.6897 - 

KeyboardInterrupt: 

In [89]:
print(Y_class.shape)
print(X.shape)

(696, 30)
(696, 103)


In [72]:
z=model.predict(X)

longprobs = np.zeros([nrows, OUTPUT_DIM])
shortprobs = np.zeros([nrows, OUTPUT_DIM])
flatprobs = np.zeros([nrows, OUTPUT_DIM])

for response in range(OUTPUT_DIM):
    for row in range(nrows):
        longprobs[row, response] = z[response][row, ISLONG]
        shortprobs[row, response] = z[response][row, ISSHORT]
        flatprobs[row, response] = z[response][row, ISFLAT]

i=0
print(flatprobs[i])
print(longprobs[i])
print(shortprobs[i])
print(flatprobs[i] + longprobs[i] + shortprobs[i])

[0.73989958 0.65319115 0.37609512 0.40323266 0.80311286 0.74617505
 0.6212942  0.64905483 0.72370356 0.59356982 0.68947989 0.47099158
 0.76194555 0.60890096 0.5783779  0.64159554 0.34791061 0.31595203
 0.50655687 0.50013244 0.5098682  0.51675969 0.55720091 0.75047684
 0.63351041 0.73421443 0.68504792 0.42430332 0.65795249 0.61809278]
[0.19224735 0.22811031 0.4466413  0.24164565 0.08169506 0.12297722
 0.18395223 0.23416008 0.12024333 0.17163867 0.1214458  0.18858837
 0.09768605 0.1967954  0.14716461 0.23285027 0.34185773 0.34976387
 0.29882261 0.34258679 0.27609351 0.18421379 0.14238672 0.0820222
 0.13043307 0.12787752 0.17103112 0.3213014  0.16271211 0.21429649]
[0.06785309 0.11869853 0.1772636  0.35512167 0.11519214 0.13084772
 0.19475353 0.11678501 0.15605308 0.23479158 0.18907437 0.34042007
 0.14036842 0.19430368 0.27445748 0.12555422 0.31023166 0.33428407
 0.19462046 0.15728082 0.21403825 0.29902649 0.30041239 0.16750097
 0.23605649 0.13790806 0.14392091 0.25439528 0.17933537 0.167

In [90]:
EPOCHS=100
BATCH_SIZE=32

def fit_predict(X, Y, model, epochs=EPOCHS, npredict=1, verbose=False):
    """for backtest, train model using Y_list v. X using n-npredict rows
    generate npredict prediction Y_list using last npredict rows of X
    if npredict=1, fit using n-1 rows, return prediction using X for final month
    if npredict=26, fit using n-26 rows, return prediction using X for final 26 months"""
    
    nrows, ncols = X.shape
    if verbose:
        print("Fit on %d rows 0 to %d" % (nrows-npredict, nrows-npredict-1))
        print("Predict on %d rows %d to %d" % (npredict, nrows-npredict, nrows-1))
        
    # keep last rows to predict against
    X_predict = X[-npredict:]
    X_predict = X_predict.reshape(npredict,X.shape[1])
    # fit on remaining rows
    X_fit = X[:-npredict]
    Y_fit = Y[:-npredict]
    
    # make a list of Ys expected by Keras
    Y_list = []
    for i in range(OUTPUT_DIM):
        Y_list.append(keras.utils.to_categorical(Y_fit[:,i], num_classes=NCLASSES))
         
    fit = model.fit(
        X_fit,
        Y_list,
        batch_size=BATCH_SIZE,
        epochs=epochs,
        verbose=0)

    Z = model.predict(X_predict)
    
    longprobs = np.zeros([npredict, OUTPUT_DIM])
    shortprobs = np.zeros([npredict, OUTPUT_DIM])
    flatprobs = np.zeros([npredict, OUTPUT_DIM])

    for response in range(OUTPUT_DIM):
        for row in range(npredict):
            longprobs[row, response] = Z[response][row, ISLONG]
            shortprobs[row, response] = Z[response][row, ISSHORT]
            flatprobs[row, response] = Z[response][row, ISFLAT]
    
    return flatprobs, longprobs, shortprobs

print("%s Start fit" % (time.strftime("%H:%M:%S")))
flatprobs, longprobs, shortprobs = fit_predict(X, Y_class, model,epochs=3,npredict=3, verbose=True)
print("%s End fit" % (time.strftime("%H:%M:%S")))

i=0
print(flatprobs[i])
print(longprobs[i])
print(shortprobs[i])
print(flatprobs[i] + longprobs[i] + shortprobs[i])

07:24:56 Start fit
Fit on 693 rows 0 to 692
Predict on 3 rows 693 to 695
07:25:03 End fit
[0.70094669 0.51267409 0.31555802 0.44605982 0.59227812 0.7188375
 0.55687535 0.50699764 0.69770986 0.37999946 0.65319031 0.36464691
 0.60252029 0.42291167 0.53504634 0.58563578 0.42592576 0.2714169
 0.46737468 0.48804164 0.47585663 0.43772307 0.41322729 0.71564108
 0.62680173 0.70730972 0.51187742 0.54305059 0.6116854  0.55277431]
[0.16946118 0.25575674 0.3817496  0.20946449 0.23679402 0.15219443
 0.25443757 0.22677046 0.16205129 0.37216562 0.18914914 0.27369422
 0.21843782 0.38384637 0.20601651 0.18339613 0.20547722 0.40441355
 0.22401495 0.25103316 0.34085706 0.32618108 0.29546684 0.13189632
 0.2010155  0.15134814 0.30073386 0.15996478 0.14782324 0.20963971]
[0.12959212 0.23156922 0.30269241 0.34447569 0.17092781 0.1289681
 0.1886871  0.26623181 0.14023887 0.24783494 0.15766062 0.36165884
 0.17904189 0.19324192 0.25893721 0.23096813 0.36859697 0.32416952
 0.30861044 0.26092517 0.18328634 0.2360

In [91]:
# 197001 = 121
STARTMONTH = 121
print(X[STARTMONTH])
print(data.iloc[STARTMONTH][:30])

[-3.34000000e+00 -1.95000000e+00 -7.59000000e+00 -7.76000000e+00
 -1.20500000e+01 -7.50000000e+00 -5.69000000e+00 -7.71000000e+00
 -7.37000000e+00 -5.26000000e+00 -9.84000000e+00 -6.31000000e+00
 -7.15000000e+00 -6.89000000e+00 -9.35000000e+00 -1.24900000e+01
 -2.34000000e+00 -7.70000000e-01 -1.21600000e+01 -4.83000000e+00
 -3.16000000e+00 -1.11700000e+01 -9.73000000e+00 -8.89000000e+00
 -8.17000000e+00 -8.28000000e+00 -6.31000000e+00 -1.31200000e+01
 -9.78000000e+00 -6.20000000e+00  5.00000000e-02  1.40000000e-01
 -8.00000000e-02  8.33333333e-02 -8.10000000e+00 -2.28666667e+00
 -2.18000000e+00 -3.36000000e+00 -7.00333333e+00 -6.82000000e+00
 -3.37666667e+00 -5.32666667e+00 -1.41000000e+00 -5.58666667e+00
 -5.43333333e+00 -6.02666667e+00 -4.45000000e+00 -4.68333333e+00
 -4.67666667e+00 -5.93666667e+00 -9.37000000e+00 -2.48000000e+00
  2.38666667e+00 -6.98666667e+00 -3.96666667e+00 -2.88333333e+00
 -4.91666667e+00 -4.43000000e+00 -4.87666667e+00 -8.43000000e+00
 -6.69666667e+00 -4.78333

In [95]:
# predict all months starting STARTMONTH
# initialize predictions matrix P

EPOCHS=500

nrows = X.shape[0]
startindex = 1000

def run_backtest(X, Y, arg_dict, startindex=0, epochs=EPOCHS, step=1, minmaxscale=False, standardscale=False):
    """create keras model; add step, to iteratively train, predict 12 months, train up to next 12 months """
    
    global P_long
    global P_short
    global P_flat
    global R 
    

    print("%s Starting backtest" % (time.strftime("%H:%M:%S")))
    count = 0
    nrows, ncols = X.shape
    P_long = np.zeros([nrows,OUTPUT_DIM])
    P_short = np.zeros([nrows,OUTPUT_DIM])
    P_flat = np.zeros([nrows,OUTPUT_DIM])

    Xscale = X.copy()
    Yscale = Y.copy()
    
    if minmaxscale:
        # minmaxscale each row (min->0, max->1) - transpose, scale, transpose back because scales by columns
        Xscale = MinMaxScaler().fit_transform(Xscale.transpose()).transpose()
        print("using MinMaxScaler")
    elif standardscale:
        # standardize each row (mean->0, SD->1)- transpose, scale, transpose back because scales by columns
        Xscale = StandardScaler().fit_transform(Xscale.transpose()).transpose()
        print("using StandardScaler")
     
    for train_index in range(startindex, nrows, step):
        # don't exceed bounds
        train_index = max(train_index, nrows-step)
        fp_index = train_index + step # eg 1000 + 26 = 1026
            
        model = build_model(n_hidden_layers = arg_dict["n_hidden_layers"],
                            hidden_layer_size = arg_dict["hidden_layer_size"], 
                            reg_penalty = arg_dict["reg_penalty"], 
                            dropout = arg_dict["dropout"],
                            verbose=arg_dict["verbose"])
        
        # fit on e.g. 0:999, predict 1000-1025
        flatprobs, longprobs, shortprobs = fit_predict(Xscale[:fp_index, :], 
                                                       Yscale[:fp_index], 
                                                       model,
                                                       epochs=epochs,
                                                       npredict=step)
        
        # store in 1000:1025 - lining up with future Xs not current X/Ys
        for i in range(step):
            if train_index + i >= nrows:
                break
            P_flat[train_index + i]= flatprobs[i]
            P_long[train_index + i]= longprobs[i]
            P_short[train_index + i]= shortprobs[i]
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training %d of %d" % (time.strftime("%H:%M:%S"), count, nrows-startindex))
            sys.stdout.flush()


In [98]:
gen(STARTMONTH)

NameError: name 'gen_returns' is not defined

In [96]:
START=121
EPOCHS=500
STEP=12
arg_dict = {"n_hidden_layers" : 2,
            "hidden_layer_size" : 32,
            "reg_penalty" : 0.0001,
            "dropout": 0.25,
            'verbose' : False
           }
     
#model = build_model(**arg_dict)
run_backtest(X, Y_class, arg_dict, startindex=START, step=STEP, epochs=EPOCHS)
#gen_returns(START)

07:37:12 Starting backtest
................................................................................
09:25:01 Still training 80 of 575
................................................................................
11:13:34 Still training 160 of 575
................................................................................
12:46:59 Still training 240 of 575
................................................................................
14:37:14 Still training 320 of 575
................................................................................
16:28:16 Still training 400 of 575
................................................................................
18:04:15 Still training 480 of 575
................................................................................
19:56:59 Still training 560 of 575
...............

IndexError: index 696 is out of bounds for axis 0 with size 696

In [ ]:
# double check results_post_LASSO
#model = LinearRegression()
#R = run_backtest(X, Y, model, coef_dict_paper, startmonth=STARTMONTH, summary=False)
results_post_LASSO = R[STARTMONTH:]
print(len(results_post_LASSO))
#print(results_post_LASSO)
print(np.mean(results_post_LASSO))
print(np.std(results_post_LASSO) * np.sqrt(12))
print(np.prod(1 + results_post_LASSO / 100))
print(np.prod(1 + results_post_LASSO / 100) ** (12.0/results_post_LASSO.shape[0]))-1

In [ ]:
# run performance chart
perf_post_LASSO = 100 * np.cumprod(1 + results_post_LASSO / 100)

def mychart(args, names=None):
    x_coords = np.linspace(1970, 2016, args[0].shape[0])
    
    plotdata = []
    for i in range(len(args)):
        tracelabel = "Trace %d" % i
        if names:
                tracelabel=names[i]
        plotdata.append(Scatter(x=x_coords,
                                y=args[i].reshape(-1),
                                mode = 'line',
                                name=tracelabel))    

    layout = Layout(
        autosize=False,
        width=600,
        height=480,
        yaxis=dict(
            type='log',
            autorange=True
        )
    )
    
    fig = Figure(data=plotdata, layout=layout)
    
    return iplot(fig)
    
mychart([perf_post_LASSO],["Post-LASSO"])

In [ ]:
# pass coef_dict as None
# fit_predict will do subset selection at each timestep using data it trains on
model = LinearRegression()
run_backtest(X, Y, model, coef_dict=None, startmonth=STARTMONTH)

In [ ]:
results_LASSO_each_timestep = R[STARTMONTH:]
perf_LASSO_each_timestep = 100 * np.cumprod(1 + results_LASSO_each_timestep / 100)
mychart([perf_LASSO_each_timestep])

In [ ]:
# pass coef_dict as 'all'
# fit_predict will use all predictors (no subset selection)
model = LinearRegression()
run_backtest(X, Y, model, coef_dict='all', startmonth=STARTMONTH)

In [ ]:
results_OLS = R[STARTMONTH:]
perf_OLS = 100 * np.cumprod(1 + results_OLS / 100)
mychart([perf_OLS])

In [ ]:
mychart([perf_post_LASSO, perf_LASSO_each_timestep, perf_OLS],["Post-LASSO", "LASSO each timestep", "OLS"])

In [ ]:
def walkforward_xval (X, Y, model, coef_dict=None):

    start = time.time()

    # generate k-folds
    n_splits = 5
    kf = KFold(n_splits=n_splits)
    kf.get_n_splits(X)
    last_indexes = []
    for train_index, test_index in kf.split(X):
        # use test_index as last index to train
        last_index = test_index[-1] + 1
        last_indexes.append(last_index)
    print("%s Generate splits %s" % (time.strftime("%H:%M:%S"), str([i for i in last_indexes])))

    print("%s Starting training" % (time.strftime("%H:%M:%S")))
    
    avg_bests = []
    for i in range(1, n_splits-1):

        models = []
        losses = []
        scores = []
        count = 0        
        # skip kfold 0 so you start with train 2x size of eval set
        last_train_index = last_indexes[i]
        last_xval_index = last_indexes[i+1]

        # set up train, xval
        # train from beginning to last_train_index        
        print("Training indexes 0 to %d" % (last_train_index-1))
        X_fit = X[:last_train_index]
        Y_fit = Y[:last_train_index]
        # xval from last_train_index to last_xval_index
        print("Cross-validating indexes %d to %d" % (last_train_index, last_xval_index -1 ))
        X_xval = X[last_train_index:last_xval_index]
        Y_xval = Y[last_train_index:last_xval_index]

        if coef_dict is None:
            print("Performing LASSO subset selection on training set")
            coef_dict = subset_selection(X_fit, Y_fit, LassoLarsIC(criterion='aic'), verbose=False)
        
        mse_list = []
        
        for response in responses:
            predcols = [predictor_reverse_dict[indstr] for indstr in coef_dict[response]]
            if len(predcols) == 0:
                continue
            responsecol = response_reverse_dict[response]
            
            fit = model.fit(X_fit[:,predcols], Y_fit[:,responsecol])
            # evaluate ... run prediction, calc MSE by industry, and average
            y_xval_pred = fit.predict(X_xval[:,predcols])
            mse_list.append(mean_squared_error(Y_xval[:,i], y_xval_pred))
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training" % (time.strftime("%H:%M:%S")))
            sys.stdout.flush()             
        # mean mse over industry ys for this fold
        xval_score = np.mean(np.array(mse_list))            

        # choose model with lowest xval loss
        print ("\n%s Xval MSE %f" % (time.strftime("%H:%M:%S"), xval_score))
        avg_bests.append(xval_score)
    
    print ("Last Xval loss %f" % (xval_score))
    # mean over folds
    avg_loss = np.mean(np.array(avg_bests))
    print ("Avg Xval loss %f" % avg_loss)
    print("--------------------------------------------------------------------------------")
    return (avg_loss, model)


In [ ]:
# get baseline using linear regression
model = LinearRegression()
walkforward_xval (X, Y, model, coef_dict=coef_dict)


In [ ]:
model = MLPRegressor(hidden_layer_sizes=(1,1,1),
                     alpha=1.0,
                     activation='tanh',
                     max_iter=10000, 
                     tol=1e-10,
                     solver='lbfgs')
walkforward_xval (X, Y, model, coef_dict=coef_dict)
# slightly better MSE than linear regression

In [ ]:
# try many combos
MODELPREFIX = "MLP"

n_hiddens = [1, 2, 3]
layer_sizes = [1, 2, 4, 8]
reg_penalties = [0.0, 0.001, 0.01, 0.1, 1]
hyperparameter_combos = list(product(n_hiddens, layer_sizes, reg_penalties))

print("%s Running %d experiments" % (time.strftime("%H:%M:%S"), len(hyperparameter_combos)))

experiments = {}

# minmaxscale each row

Xscale = X.copy()
Yscale = Y.copy()

for i in range(Xscale.shape[0]):
    Xscale[i] = Xscale[i] - np.min(Xscale[i])
    Xscale[i] = Xscale[i]/np.max(Xscale[i])


for i in range(Yscale.shape[0]):
    Yscale[i] = Yscale[i] - np.min(Yscale[i])
    Yscale[i] = Yscale[i]/np.max(Yscale[i])
        
for counter, param_list in enumerate(hyperparameter_combos):
    n_hidden_layers, layer_size, reg_penalty = param_list
    print("%s Running experiment %d of %d" % (time.strftime("%H:%M:%S"), counter+1, len(hyperparameter_combos)))
    key = (n_hidden_layers, layer_size, reg_penalty)
    print("%s n_hidden_layers = %d, hidden_layer_size = %d, reg_penalty = %.6f" % 
          (time.strftime("%H:%M:%S"), n_hidden_layers, layer_size, reg_penalty))
    hls = tuple([layer_size]*n_hidden_layers)
    model = MLPRegressor(hidden_layer_sizes=hls,
                         alpha=reg_penalty,
                         activation='tanh',
                         max_iter=10000, 
                         tol=1e-10,
                         solver='lbfgs')
    
    score, model = walkforward_xval (X, Y, model, coef_dict=coef_dict)

    experiments[key] = score


In [ ]:
# list and chart experiments
flatlist = [list(l[0]) + [l[1]] for l in experiments.items()]
 
lossframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "loss"])
lossframe.sort_values(['loss'])

In [ ]:
# we can pick lowest loss , but first we look at patterns by hyperparameter
pd.DataFrame(lossframe.groupby(['n_hidden_layers'])['loss'].mean())


In [ ]:
pd.DataFrame(lossframe.groupby(['layer_size'])['loss'].mean())


In [ ]:
pd.DataFrame(lossframe.groupby(['reg_penalty'])['loss'].mean())


In [ ]:
def plot_matrix(lossframe, x_labels, y_labels, x_suffix="", y_suffix=""):

    pivot = lossframe.pivot_table(index=[x_labels], columns=[y_labels], values=['loss'])
    # specify labels as strings, to force it to use a discrete axis
    if lossframe[x_labels].dtype == np.float64 or lossframe[x_labels].dtype == np.float32:
        xaxis = ["%f %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    else:
        xaxis = ["%d %s" % (i, x_suffix) for i in pivot.columns.levels[1].values]
    if lossframe[y_labels].dtype == np.float64 or lossframe[y_labels].dtype == np.float32:
        yaxis = ["%f %s" % (i, y_suffix) for i in pivot.index.values]
    else:
        yaxis = ["%d %s" % (i, y_suffix) for i in pivot.index.values]
        
    print(xaxis, yaxis)
    """plot a heat map of a matrix"""
    chart_width=640
    chart_height=480
    
    layout = Layout(
        title="%s v. %s" % (x_labels, y_labels),
        height=chart_height,
        width=chart_width,     
        margin=dict(
            l=150,
            r=30,
            b=120,
            t=100,
        ),
        xaxis=dict(
            title=y_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
        yaxis=dict(
            title=x_labels,
            tickfont=dict(
                family='Arial, sans-serif',
                size=10,
                color='black'
            ),
        ),
    )
    
    data = [Heatmap(z=pivot.values,
                    x=xaxis,
                    y=yaxis,
                    colorscale=[[0, 'rgb(0,0,255)', [1, 'rgb(255,0,0)']]],
                   )
           ]

    fig = Figure(data=data, layout=layout)
    return iplot(fig, link_text="")

plot_matrix(lossframe, "n_hidden_layers", "layer_size", x_suffix=" units", y_suffix=" layers")



In [ ]:
plot_matrix(lossframe, "n_hidden_layers", "reg_penalty", x_suffix="p", y_suffix=" layers")


In [ ]:
plot_matrix(lossframe, "reg_penalty", "layer_size", x_suffix=" units", y_suffix="p")


In [ ]:
# these results are not very good, ~same as LinearRegression, but try best one
# 1-unit layers is not really a NN but anyway let's see how it does
model = MLPRegressor(hidden_layer_sizes=(1,1,1),
                     alpha=0.01,
                     activation='tanh',
                     max_iter=10000, 
                     tol=1e-10,
                     solver='lbfgs')
run_backtest(X, Y, model, startmonth=STARTMONTH, minmaxscale=False)
# slightly better OOS MSE than linear regression (45.2) but worse Sharpe

In [ ]:
# try keras instead of sklearn MLPRegressor
INPUT_DIM = X.shape[1]
print(INPUT_DIM)
OUTPUT_DIM = len(responses) # 30

def build_model(n_hidden_layers = 2,
                hidden_layer_size = 32,
                reg_penalty = 0.0001,
                dropout = 0.333,
                verbose=True):

    main_input = Input(shape=(INPUT_DIM,), 
                       dtype='float32', 
                       name='main_input')
    lastlayer=main_input

    for i in range(n_hidden_layers):
        if verbose:
            print("layer %d size %d, reg_penalty %.8f, dropout %.3f" % (i, hidden_layer_size, reg_penalty, dropout))
        lastlayer = Dense(units = hidden_layer_size, 
                          activation = 'relu',
                          kernel_initializer = keras.initializers.glorot_uniform(),
                          kernel_regularizer=keras.regularizers.l1(reg_penalty),
                          name = "Dense%02d" % i)(lastlayer)

        if dropout:
            lastlayer = Dropout(dropout, name = "Dropout%02d" % i)(lastlayer)
    
    outputs = []
    for i in range(OUTPUT_DIM):
        # OUTPUT_DIM outputs
        output01 = Dense(1,
                         activation='linear', 
                         name='output%02d' % i)(lastlayer)
        outputs.append(output01)
    
    model = Model(inputs=[main_input], outputs=outputs)
    if verbose:
        print(model.summary())
    model.compile(loss="mse", optimizer="rmsprop", loss_weights=[1.]*OUTPUT_DIM)
    return model


In [ ]:
# run an experiment with walk-forward cross-validation

EPOCHS = 500
#VAL_SPLIT = 0.2
BATCH_SIZE = 128

def run_experiment (n_hidden_layers = 2,
                    hidden_layer_size = 8,
                    reg_penalty = 0.0,
                    dropout = 0.5,
                    epochs = EPOCHS
                   ):

    start = time.time()

    # generate k-folds
    n_splits = 5
    kf = KFold(n_splits=n_splits)
    kf.get_n_splits(X)
    last_indexes = []
    for train_index, test_index in kf.split(X):
        # use test_index as last index to train
        last_index = test_index[-1] + 1
        last_indexes.append(last_index)

    print("%s Generate splits %s" % (time.strftime("%H:%M:%S"), str([i for i in last_indexes])))
    
    avg_bests = []

    print("%s Build model" % (time.strftime("%H:%M:%S")))
    model = build_model(n_hidden_layers = n_hidden_layers,
                        hidden_layer_size = hidden_layer_size,
                        reg_penalty = reg_penalty,
                        dropout = dropout)
    print("Compile time : %s" % str(time.time() - start))
    print("Starting to train : %s" % (time.strftime("%H:%M:%S")))
    for i in range(1, n_splits-1):

        models = []
        losses = []
        scores = []
        count = 0        
        # skip kfold 0 so you start with train 2x size of eval set
        last_train_index = last_indexes[i]
        last_xval_index = last_indexes[i+1]

        # set up train, xval
        # train from beginning to last_train_index
        print("Training indexes 0 to %d" % (last_train_index-1))
        X_fit = X[:last_train_index]
        Y_fit = Y[:last_train_index]
        # xval from last_train_index to last_xval_index
        print("Cross-validating indexes %d to %d" % (last_train_index, last_xval_index -1 ))
        X_xval = X[last_train_index:last_xval_index]
        Y_xval = Y[last_train_index:last_xval_index]

        responses = []
        for i in range(OUTPUT_DIM):
            responses.append(Y_fit[:,i])
        # train for epochs
        for epoch in range(epochs):
            fit = model.fit(
                X_fit,
                responses,
                batch_size=BATCH_SIZE,
                #validation_split=VAL_SPLIT,
                epochs=1,
                verbose=0)
            
            train_loss = fit.history['loss'][-1]
            # evaluate ... run prediction, calc MSE by industry, and average
            y_xval_pred = np.array(model.predict(X_xval))
            y_xval_pred = y_xval_pred.reshape(Y_xval.T.shape)
            y_xval_pred = y_xval_pred.T
            mse_list = []
            for i in range(len(industries)):
                mse_list.append(mean_squared_error(Y_xval[:,i], y_xval_pred[:,i]))
            xval_score = np.mean(np.array(mse_list))            
            
            losses.append(train_loss)
            scores.append(xval_score)
            models.append(copy.copy(model))

            bestloss_index = np.argmin(scores)
            bestloss_value = scores[bestloss_index]

            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training %d of %d" % (time.strftime("%H:%M:%S"), count, epochs))
                
            sys.stdout.flush()            
            
            # stop if loss rises by 20% from best
            if xval_score / bestloss_value > 1.2:
                print("Stopping early" )
                break

        # choose model with lowest xval loss
        print("")
        print ("%s Best Xval loss epoch %d, value %f" % (time.strftime("%H:%M:%S"), bestloss_index, bestloss_value))
        avg_bests.append(bestloss_value)
        model = models[bestloss_index]
    
    print ("Last Xval loss %f" % (bestloss_value))
    avg_loss = np.mean(np.array(avg_bests))
    print ("Avg Xval loss %f" % avg_loss)
    print("--------------------------------------------------------------------------------")
    return (avg_loss, model)


In [ ]:
run_experiment()

In [ ]:
# notable improvement in MSE vs. LinearRegression
# run a lot of experiments in big xval loop to pick best hyperparameters

MODELPREFIX = "FFNN"

n_hiddens = [1, 2, 3]
layer_sizes = [2, 4, 8, 16]
reg_penalties = [0.0, 0.0001, 0.001, 0.01]
dropouts = [0.25]

hyperparameter_combos = list(product(n_hiddens, layer_sizes, reg_penalties, dropouts))

print("%s Running %d experiments" % (time.strftime("%H:%M:%S"), len(hyperparameter_combos)))

experiments = {}

for counter, param_list in enumerate(hyperparameter_combos):
    n_hidden_layers, layer_size, reg_penalty, dropout = param_list
    print("%s Running experiment %d of %d" % (time.strftime("%H:%M:%S"), counter+1, len(hyperparameter_combos)))
    key = (n_hidden_layers, layer_size, reg_penalty, dropout)
    score, model = run_experiment(n_hidden_layers = n_hidden_layers,
                                  hidden_layer_size = layer_size,
                                  reg_penalty = reg_penalty,
                                  dropout = dropout,
                                  epochs=EPOCHS)
    experiments[key] = score 
    modelname = "%s_%.6f_%d_%d_%.6f_%.3f" % (MODELPREFIX, score, n_hidden_layers, layer_size, reg_penalty, dropout)
    print("%s Saving %s.h5" % (time.strftime("%H:%M:%S"), modelname))
    model.save("%s.h5" % modelname)
    model.save_weights("%s_weights.h5" % modelname)


In [ ]:
# list and chart experiments
flatlist = [list(l[0]) + [l[1]] for l in experiments.items()]

lossframe = pd.DataFrame(flatlist, columns=["n_hidden_layers", "layer_size", "reg_penalty", "dropout",
                                            "loss"])
lossframe.sort_values(['loss'])
# better than LinearRegression or MLPRegressor

In [ ]:
# we can pick lowest loss , but first we look at patterns by hyperparameter
pd.DataFrame(lossframe.groupby(['n_hidden_layers'])['loss'].mean())


In [ ]:
pd.DataFrame(lossframe.groupby(['layer_size'])['loss'].mean())

In [ ]:
pd.DataFrame(lossframe.groupby(['reg_penalty'])['loss'].mean())

In [ ]:
plot_matrix(lossframe, "n_hidden_layers", "layer_size", x_suffix=" units", y_suffix=" layers")


In [ ]:
plot_matrix(lossframe, "n_hidden_layers", "reg_penalty", x_suffix="p", y_suffix=" layers")

In [ ]:
plot_matrix(lossframe, "reg_penalty", "layer_size", x_suffix=" units", y_suffix="p")


In [ ]:
EPOCHS=500

def fit_predict(X, Y, model, epochs=EPOCHS, npredict=1, verbose=False):
    """for backtest, train model using Y_list v. X using n-npredict rows
    generate npredict prediction Y_list using last npredict rows of X
    if npredict=1, fit using n-1 rows, return prediction using X for final month
    if npredict=26, fit using n-26 rows, return prediction using X for final 26 months"""
    
    nrows = X.shape[0]
    if verbose:
        print("Fit on %d rows 0 to %d" % (nrows-npredict, nrows-npredict-1))
        print("Predict on %d rows %d to %d" % (npredict, nrows-npredict, nrows-1))
        
    # keep last rows to predict against
    X_predict = X[-npredict:]
    X_predict = X_predict.reshape(npredict,X.shape[1])
    # fit on remaining rows
    X_fit = X[:-npredict]
    Y_fit = Y[:-npredict]
    
    # make a list of Ys expected by Keras
    Y_list = []
    for i in range(OUTPUT_DIM):
        Y_list.append(Y_fit[:,i])
        
    fit = model.fit(
        X_fit,
        Y_list,
        batch_size=BATCH_SIZE,
        epochs=epochs,
        verbose=0)
    
    Z = model.predict(X_predict)
    # get back a list of ncols arrays, reshape each to 1D 1 x npredict array
    Z = [z.reshape(npredict) for z in Z]
    # return npredict x ncols array
    return np.array(Z).transpose()

print("%s Start fit" % (time.strftime("%H:%M:%S")))
predictions = fit_predict(X, Y, model,epochs=3,npredict=3, verbose=True)
print("%s End fit" % (time.strftime("%H:%M:%S")))

predictions

In [ ]:
EPOCHS=500

nrows = X.shape[0]
startindex = 1000

def run_backtest(X, Y, arg_dict, startindex=0, epochs=EPOCHS, step=1, minmaxscale=False, standardscale=False):
    """create keras model; add step, to iteratively train, predict 12 months, train up to next 12 months """
    global P
    global R 
    
    print("%s Starting backtest" % (time.strftime("%H:%M:%S")))
    P = np.zeros((Y.shape[0],OUTPUT_DIM))
    
    count = 0
    nrows = X.shape[0]

    Xscale = X.copy()
    Yscale = Y.copy()
    
    if minmaxscale:
        # minmaxscale each row (min->0, max->1) - transpose, scale, transpose back because scales by columns
        Xscale = MinMaxScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = MinMaxScaler().fit_transform(Yscale.transpose()).transpose()
        print("using MinMaxScaler")
    elif standardscale:
        # standardize each row (mean->0, SD->1)- transpose, scale, transpose back because scales by columns
        Xscale = StandardScaler().fit_transform(Xscale.transpose()).transpose()
        Yscale = StandardScaler().fit_transform(Yscale.transpose()).transpose()
        print("using StandardScaler")
     
    for train_index in range(startindex, nrows, step):
        if train_index + step >= nrows:
            train_index = nrows-step
            
        model = build_model(n_hidden_layers = arg_dict["n_hidden_layers"],
                            hidden_layer_size = arg_dict["hidden_layer_size"], 
                            reg_penalty = arg_dict["reg_penalty"], 
                            dropout = arg_dict["dropout"],
                            verbose=arg_dict["verbose"])
        
        fp_index = train_index + step # eg 1000 + 26 = 1026

        # fit on e.g. 0:999, predict 1000-1025
        predictions = fit_predict(Xscale[:fp_index, :], 
                                  Yscale[:fp_index], 
                                  model,
                                  epochs=epochs,
                                  npredict=step)
        # store in 1000:1025 - lining up with future Xs not current X/Ys
        for i in range(step):
            P[train_index + i]= predictions[i]
            sys.stdout.write('.')
            count += 1
            if count % 80 == 0:
                print("")
                print("%s Still training %d of %d" % (time.strftime("%H:%M:%S"), count, nrows-startindex))
            sys.stdout.flush()

    msetemp = (P-Yscale)**2
    mse = np.mean(np.nan_to_num(msetemp))
    print("MSE across all predictions: %.4f" % mse)


In [ ]:
def gen_returns(startindex):
    # generate returns
    global X
    global Y
    global P
    global R

    msetemp = (P[startindex+1:]-Y[startindex:-1])**2
#    msetemp = (P[startindex:]-Y[startindex:])**2
    mse = np.mean(np.nan_to_num(msetemp))
    print("MSE across all predictions: %.4f" % mse)
    print("Variance: %.4f" % (np.mean(Y[startindex:]**2)))
    print("R-squared: %.4f" % (1- mse/np.mean(Y[startindex:]**2)))

    
    nrows = P.shape[0]

    R = np.zeros(nrows)
    NUM_POSITIONS = 6 # top quintile (and bottom)
    
    for train_index in range(startindex, nrows):
        # get indexes, sorted smallest to largest
        select_array = np.argsort(P[train_index])
#        print(P[train_index, select_array])
#        print(Y[train_index, select_array])
#        print("--")
        # leftmost 6
        short_indexes = select_array[:NUM_POSITIONS]
        # rightmost 6
        long_indexes = select_array[-NUM_POSITIONS:]
        # compute equal weighted long/short return
        # + 50% long * 0.25 * perf of long indexes
        R[train_index] = R[train_index] + 0.50 * np.mean(X[train_index, long_indexes])
        # - 50% short * 0.25 * perf of short indexes
        R[train_index] = R[train_index] - 0.50 * np.mean(X[train_index, short_indexes])
                
    # truncate to nonzero part of R            
    results = R[startindex:]
    
    index = pd.date_range(data.iloc[START].name,periods=results.shape[0], freq='M')
    perfdata = pd.DataFrame(results,index=index,columns=['Returns'])
    perfdata['Equity'] = 100 * np.cumprod(1 + results / 100)
    
    stats = perfdata['Equity'].calc_stats()
    
    retframe = pd.DataFrame([stats.stats.loc['start'],
                             stats.stats.loc['end'],
                             stats.stats.loc['cagr'],
                             stats.stats.loc['yearly_vol'],
                             stats.stats.loc['yearly_sharpe'],
                             stats.stats.loc['max_drawdown'],
                             ffn.core.calc_sortino_ratio(perfdata.Returns, rf=0, nperiods=results.shape[0], annualize=False),
                            ],
                            index = ['start',
                                     'end',
                                     'cagr',
                                     'yearly_vol',
                                     'yearly_sharpe',
                                     'max_drawdown',
                                     'sortino',
                                    ],
                            columns=['Value'])
    return retframe



In [ ]:
gen_returns(START)

In [ ]:
START=121
EPOCHS=500
STEP=12
arg_dict = {"n_hidden_layers" : 3,
            "hidden_layer_size" : 4,
            "reg_penalty" : 0.0,
            "dropout": 0.25,
            'verbose' : False
           }
     
#model = build_model(**arg_dict)
run_backtest(X, Y, arg_dict, startindex=START, step=STEP, epochs=EPOCHS)
gen_returns(START)